In [1]:
!unzip data/data202366/LEVIR-CD.zip -d PaddleCD/data

  inflating: PaddleCD/data/train/B/train_179.png  
  inflating: PaddleCD/data/train/B/train_18.png  
  inflating: PaddleCD/data/train/B/train_180.png  
  inflating: PaddleCD/data/train/B/train_181.png  
  inflating: PaddleCD/data/train/B/train_182.png  
  inflating: PaddleCD/data/train/B/train_183.png  
  inflating: PaddleCD/data/train/B/train_184.png  
  inflating: PaddleCD/data/train/B/train_185.png  
  inflating: PaddleCD/data/train/B/train_186.png  
  inflating: PaddleCD/data/train/B/train_187.png  
  inflating: PaddleCD/data/train/B/train_188.png  
  inflating: PaddleCD/data/train/B/train_189.png  
  inflating: PaddleCD/data/train/B/train_19.png  
  inflating: PaddleCD/data/train/B/train_190.png  
  inflating: PaddleCD/data/train/B/train_191.png  
  inflating: PaddleCD/data/train/B/train_192.png  
  inflating: PaddleCD/data/train/B/train_193.png  
  inflating: PaddleCD/data/train/B/train_194.png  
  inflating: PaddleCD/data/train/B/train_195.png  
  inflating: PaddleCD/data/train/

In [2]:
import numpy as np
import cv2
import os

def replace(path, save_path):
    path_list  = os.listdir(path)
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    for item in path_list:
        try:
            image = cv2.imread(os.path.join(path, item))[:, :, 0]
            image[image <= 127] = 0
            image[image > 127] = 1
            cv2.imwrite(os.path.join(save_path, item), image)
        except:
            print("error!%s不存在" % item)

replace("PaddleCD/data/train/label", "PaddleCD/data/train/target")
replace("PaddleCD/data/val/label", "PaddleCD/data/val/target")
replace("PaddleCD/data/test/label", "PaddleCD/data/test/target")

In [3]:
import os
import cv2
import numpy as np
from tqdm import tqdm


def split_image(image_root, aim_root, split_size=(512, 512), stride=(256, 256), scale=(0.5, 1)):
    image_list = os.listdir(os.path.join(image_root, "target"))
    if not os.path.exists(aim_root):
        os.makedirs(os.path.join(aim_root, "A"))
        os.makedirs(os.path.join(aim_root, "B"))
        os.makedirs(os.path.join(aim_root, "target"))
    for image in tqdm(image_list):
        image_name = image.split(".")[0]
        image_a = cv2.imread(os.path.join(image_root, "A", image))
        image_b = cv2.imread(os.path.join(image_root, "B", image))
        target = cv2.imread(os.path.join(image_root, "target", image))
        for k in scale:
            scale_a = cv2.resize(image_a, (int(image_a.shape[1] * k), int(image_a.shape[0] * k)), interpolation=cv2.INTER_LINEAR)
            scale_b = cv2.resize(image_b, (int(image_b.shape[1] * k), int(image_b.shape[0] * k)), interpolation=cv2.INTER_LINEAR)
            scale_t = cv2.resize(target, (int(target.shape[1] * k), int(target.shape[0] * k)), interpolation=cv2.INTER_NEAREST)
            x_list = int((scale_t.shape[0] - (split_size[0] - stride[0])) / stride[0])
            y_list = int((scale_t.shape[1] - (split_size[1] - stride[1])) / stride[1])
            for x in range(x_list):
                for y in range(y_list):
                    cv2.imwrite(os.path.join(aim_root, "A", image_name + "_%d_%d_%d.png" % (k*100, x, y)),
                                 scale_a[x*stride[0]:x*stride[0]+split_size[0], y*stride[1]:y*stride[1]+split_size[1]])
                    cv2.imwrite(os.path.join(aim_root, "B", image_name + "_%d_%d_%d.png" % (k*100, x, y)),
                                 scale_b[x*stride[0]:x*stride[0]+split_size[0], y*stride[1]:y*stride[1]+split_size[1]])
                    cv2.imwrite(os.path.join(aim_root, "target", image_name + "_%d_%d_%d.png" % (k*100, x, y)),
                                 scale_t[x*stride[0]:x*stride[0]+split_size[0], y*stride[1]:y*stride[1]+split_size[1], 0])

split_image("PaddleCD/data/train", "PaddleCD/data/train_split")
split_image("PaddleCD/data/val", "PaddleCD/data/val_split")
split_image("PaddleCD/data/test", "PaddleCD/data/test_split")

100%|██████████| 128/128 [00:45<00:00,  2.84it/s]


In [ ]:
# 框架准备
!cd PaddleCD
!pip install -r requirements.txt --user
!pip install -v -e .            # 修改了代码需要从新运行这一行，如果改配置文件就不需要
!python setup.py install

In [ ]:
# 模型训练
python tools/train.py --config configs/CD/UperNet_Swin_Tiny_CD_Sub.yml --save_interval 800 --use_vdl --save_dir output/UperNet_Sub --log_iters 50 --num_worker 2 --do_eval --precision fp16 --amp_level O1

In [ ]:
# 模型验证
python tools/val.py --config configs/CD/UperNet_Swin_Tiny_CD_Sub.yml --model_path output/UperNet_Sub/best_model/model.pdparams --aug_eval --scales 0.75 1.0 1.25 --flip_horizontal --is_slide --crop_size 512 512 --stride 256 256